In [18]:
import pandas as pd
import pyodbc
import psycopg2
import time 
import datetime

In [19]:
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password)

In [20]:
query = """SELECT a.id,
           active_user_id,
           REGEXP_REPLACE(a.active_user_phone,'^84','0') as user_phone,
           code as employee_phone,
           e.email as employee_email,
           concat(e.last_name,' ',e.first_name) as employee_name,
           to_char(date(a.created_at),'yyyy-mm-dd') as invoice_created_date,
           code,
           case
               when cast(a.created_at as date) >= cast(s.created_at as date) then 'Đã Đăng Ký'
               else 'Chưa Đăng Ký'
                   end as Status,
           sum(count) as total_view,
           sum(duration) as total_duration
    FROM payment_report.pnc_invoices a
    left join payment_report.pnc_employees e on e.phone = a.code
    left join ( select "userIdSysMap",
                       status,
                       phone,
                       created_at,
                       user_id,
                       type,
                       count,
                       "Date" from payment_report.code_demo_view
    ) b
    ON active_user_id = "userIdSysMap" and b."Date" >= to_char(date(a.created_at),'yyyy-mm-dd')
    left join ( select user_id,
                       sum(duration) as duration
                from payment_report.duration_code_demo
                group by user_id) c
    on a.active_user_id = c.user_id
    left join payment_report.sub_customers S on S.user_id = a.active_user_id
    Where serial = 'IBB' and active_plan_id IN (3290)
    group by a.id,active_user_id,a.created_at,duration,e.first_name,e.last_name,email,e.phone,s.created_at,code
    order by invoice_created_date desc"""

In [21]:
postgre = pd.read_sql(query,conn)

In [22]:
postgre = postgre.fillna(0)

In [23]:
postgre = postgre.astype({"total_view":"int","total_duration":"int"})

In [24]:
result = postgre[(postgre['total_view']>=20)&(postgre['status'] == 'Chưa Đăng Ký')]

In [25]:
result = result[['id','active_user_id','user_phone','employee_phone','employee_email','employee_name','invoice_created_date']]

In [26]:
a = datetime.datetime.now()
a = datetime.datetime.strftime(a,"%Y-%m-%d")

In [29]:
server = server
database = database
username = username
password = password
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [30]:
query = """select * from OTT_Code_Demo"""

In [31]:
data = pd.read_sql(query,cnxn)

In [33]:
data

,id,active_user_id,user_phone,employee_phone,employee_email,employee_name,invoice_created_date,updated_at
0,693,27246696,0943915903,0865087383,quantm3@fpt.com.vn,Tran Minh Quan,2022-06-14,2022-06-14
1,693,27246696,0943915903,0865087383,thuydb5@fpt.com.vn,Duong Bich Thuy,2022-06-14,2022-06-14
2,693,27246696,0943915903,0865087383,manntm4@fpt.com.vn,Nguyen Thi Minh Man,2022-06-14,2022-06-14
3,69256,29972223,0904066364,0977022591,HanhPT12@fpt.com.vn,Phùng Th? H?nh,2022-06-13,2022-06-14
4,69240,13106993,0929222095,0352003094,Huyennt129@fpt.com.vn,Huy?n Nguy?n Th?,2022-06-13,2022-06-14
...,...,...,...,...,...,...,...,...
14582,693,27246696,0943915903,0865087383,longth17@fpt.com.vn,Tran Hoang Long,2022-06-14,2022-06-14
14583,788,23500780,0399568943,0862469688,tuanlq21@fpt.com.vn,Lâm Qu?c Tu?n,2021-09-26,2022-06-14
14584,697,7255661,0988032759,0988032759,Thanhlv32@fpt.com.vn,Lê Van Thành,2021-09-25,2022-06-14
14585,692,27133227,0904568385,0359487408,NamNT58@fpt.com.vn,Nguy?n Ti?n Nam Nam,2021-09-25,2022-06-14


In [40]:
data[data['id'] == 69240]

,id,active_user_id,user_phone,employee_phone,employee_email,employee_name,invoice_created_date,updated_at
4,69240,13106993,0929222095,0352003094,Huyennt129@fpt.com.vn,Huy?n Nguy?n Th?,2022-06-13,2022-06-14


In [34]:
list_id = data['id'].to_list()

In [38]:
len(list_id)

14587

In [35]:
result = result[~result['id'].isin(list_id)]

In [36]:
result

,id,active_user_id,user_phone,employee_phone,employee_email,employee_name,invoice_created_date,updated_at
22225,47288,18666174,0788350789,0788350789,luanta2@fpt.com.vn,Trần Anh Luân,2022-03-17,2022-06-15
44236,25486,18027032,0337921098,0978432998,DuyPT28@fpt.com.vn,Duy Phạm Tuấn,2022-01-20,2022-06-15


In [27]:
result['updated_at'] = a

In [28]:
result.head()

,id,active_user_id,user_phone,employee_phone,employee_email,employee_name,invoice_created_date,updated_at
55,69240,13106993,0929222095,0352003094,Huyennt129@fpt.com.vn,Huyền Nguyễn Thị,2022-06-13,2022-06-15
97,69256,29972223,0904066364,0977022591,HanhPT12@fpt.com.vn,Phùng Thị Hạnh,2022-06-13,2022-06-15
107,69203,3865804,0372331799,0941404582,Dongtv6@fpt.com.vn,Trương Văn Đông,2022-06-12,2022-06-15
115,69236,6430459,0962742730,0968178998,Duongnd4@fpt.com.vn,Dương Nguyễn Đức,2022-06-12,2022-06-15
131,69219,30221741,0937150982,0988795663,thanhnm30@fpt.com.vn,Nguyễn Minh Thành,2022-06-12,2022-06-15


In [42]:
i=0
sql = "SET IDENTITY_INSERT OTT_Code_Demo ON INSERT INTO" + " " + "OTT_Code_Demo" + " " + "(" + 'id'+ ',' + " " + 'active_user_id'+ ',' + " " + 'user_phone' + ","+ "employee_phone"+ ',' + " " + 'employee_email'+ ',' + " " + 'employee_name' + ","+ "invoice_created_date"+ ","+ "updated_at"+ ")" + " " +'VALUES'+ ' ' + '(' +"'"+ str(result['id'].to_list()[i]) + "'"+ ',' + "'" +str(result['active_user_id'].to_list()[i]) +  "'" + ',' +"'" + str(result['user_phone'].to_list()[i]) + "'" + ',' +"'" + str(result['employee_phone'].to_list()[i]) +"'" +"," + "'"+str(result['employee_email'].to_list()[i]) + "'"+ ',' +'N'+ "'" +str(result['employee_name'].to_list()[i]) +  "'" + ',' +"'" + str(result['invoice_created_date'].to_list()[i]) + "'"+ ',' +"'" + str(result['updated_at'].to_list()[i]) + "'" + ')'  


In [43]:
sql

"SET IDENTITY_INSERT OTT_Code_Demo ON INSERT INTO OTT_Code_Demo (id, active_user_id, user_phone,employee_phone, employee_email, employee_name,invoice_created_date,updated_at) VALUES ('47288','18666174','0788350789','0788350789','luanta2@fpt.com.vn','Trần Anh  Luân','2022-03-17','2022-06-15')"

In [34]:
for i in range(len(result)):
    server = server
    database = database
    username = username
    password = password
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    sql = "SET IDENTITY_INSERT OTT_Code_Demo ON INSERT INTO" + " " + "OTT_Code_Demo" + " " + "(" + 'id'+ ',' + " " + 'active_user_id'+ ',' + " " + 'user_phone' + ","+ "employee_phone"+ ',' + " " + 'employee_email'+ ',' + " " + 'employee_name' + ","+ "invoice_created_date"+ ","+ "updated_at"+ ")" + " " +'VALUES'+ ' ' + '(' +"'"+ str(result['id'].to_list()[i]) + "'"+ ',' + "'" +str(result['active_user_id'].to_list()[i]) +  "'" + ',' +"'" + str(result['user_phone'].to_list()[i]) + "'" + ',' +"'" + str(result['employee_phone'].to_list()[i]) +"'" +"," + "'"+str(result['employee_email'].to_list()[i]) + "'"+ ',' + "'" +str(result['employee_name'].to_list()[i]) +  "'" + ',' +"'" + str(result['invoice_created_date'].to_list()[i]) + "'"+ ',' +"'" + str(result['updated_at'].to_list()[i]) + "'" + ')'  
    #print(sql)
    cursor.execute(sql)
    cnxn.commit()
    #time.sleep(0.1)

In [ ]:
cnxn.close()

In [42]:
server = server
database = database
username = username
password = password
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
sql = "INSERT INTO" + " " + "OTT_Code_Demo" + " " + "(" + 'id'+ ',' + " " + 'active_user_id'+ ',' + " " + 'user_phone' + ","+ "employee_phone"+ ',' + " " + 'employee_email'+ ',' + " " + 'employee_name' + ","+ "invoice_created_date"+ ","+ "updated_at"+ ")" + " " +'VALUES'+ ' ' + '(' +"'"+ '0855606043' + "'"+ ',' + "'" +'0855606043' +  "'" + ',' +"'" + '0855606043' + "'" + ',' +"'" + '0855606043' +"'" +"," + "'" + 'longth17@fpt.com.vn' + "'"+ ',' +'N'+ "'" +'Tran Hoang Long' +  "'" + ',' +"'" + '2022-06-14' + "'"+ ',' +"'" + '2022-06-14' + "'" + ')'
print(sql)
cursor.execute(sql)
cnxn.commit()

INSERT INTO OTT_Code_Demo (id, active_user_id, user_phone,employee_phone, employee_email, employee_name,invoice_created_date,updated_at) VALUES ('693','27246696','0943915903','0865087383','longth17@fpt.com.vn','Tran Hoang Long','2022-06-14','2022-06-14')


In [1]:
sql = "INSERT INTO" + " " + "OTT_Code_Demo" + " " + "(" + 'id'+ ',' + " " + 'active_user_id'+ ',' + " " + 'user_phone' + ","+ "employee_phone"+ ',' + " " + 'employee_email'+ ',' + " " + 'employee_name' + ","+ "invoice_created_date"+ ","+ "updated_at"+ ")" + " " +'VALUES'+ ' ' + '(' +"'"+ '0855606043' + "'"+ ',' + "'" +'0855606043' +  "'" + ',' +"'" + '0855606043' + "'" + ',' +"'" + '0855606043' +"'" +"," + "'" + 'longth17@fpt.com.vn' + "'"+ ',' +'N'+ "'" +'Tran Hoang Long' +  "'" + ',' +"'" + '2022-06-14' + "'"+ ',' +"'" + '2022-06-14' + "'" + ')'
print(sql)

INSERT INTO OTT_Code_Demo (id, active_user_id, user_phone,employee_phone, employee_email, employee_name,invoice_created_date,updated_at) VALUES ('0855606043','0855606043','0855606043','0855606043','longth17@fpt.com.vn',N'Tran Hoang Long','2022-06-14','2022-06-14')
